# 读取 Pick Blue Bottle HDF5 文件数据

这个 notebook 用于读取和分析 `rosbag2_2026_01_09-21_24_48_0.h5` 文件中的所有数据。

## 文件信息
- **文件路径**: `pick_blue_bottle/rosbag2_2026_01_09-21_24_48/rosbag2_2026_01_09-21_24_48_0.h5`
- **数据来源**: ROS2 bag 文件转换后的 HDF5 格式
- **用途**: 包含关节状态、图像等机器人数据


In [2]:
import h5py
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
import cv2
from PIL import Image
from io import BytesIO

# 设置文件路径
hdf5_path = Path("/home/wyz/realsense_ws/data1//rosbag2_2026_01_09-21_24_48/rosbag2_2026_01_09-21_24_48_0.h5")

print(f"📁 文件路径: {hdf5_path}")
print(f"📊 文件是否存在: {hdf5_path.exists()}")

if hdf5_path.exists():
    file_size = hdf5_path.stat().st_size / (1024 * 1024)  # MB
    print(f"📦 文件大小: {file_size:.2f} MB")


📁 文件路径: /home/wyz/realsense_ws/data1/rosbag2_2026_01_09-21_24_48/rosbag2_2026_01_09-21_24_48_0.h5
📊 文件是否存在: True
📦 文件大小: 8.91 MB


## 1. 探索 HDF5 文件结构


In [3]:
def print_hdf5_structure(name, obj, indent=0):
    """递归打印 HDF5 文件结构"""
    prefix = "  " * indent
    if isinstance(obj, h5py.Dataset):
        shape_str = " x ".join(map(str, obj.shape))
        print(f"{prefix}📄 {name.split('/')[-1]}: Dataset")
        print(f"{prefix}   Shape: ({shape_str})")
        print(f"{prefix}   Dtype: {obj.dtype}")
        if obj.size > 0 and obj.size < 10:
            print(f"{prefix}   Sample: {obj[:]}")
    else:
        print(f"{prefix}📁 {name.split('/')[-1]}: Group")
        if hasattr(obj, 'attrs') and len(obj.attrs) > 0:
            print(f"{prefix}   Attributes: {dict(obj.attrs)}")

# 打开文件并打印结构
with h5py.File(hdf5_path, 'r') as f:
    print("=" * 80)
    print("📂 HDF5 文件结构")
    print("=" * 80)
    print(f"\n顶层键: {list(f.keys())}\n")
    
    # 打印完整结构
    f.visititems(print_hdf5_structure)
    
    # 打印属性（如果有）
    if hasattr(f, 'attrs') and len(f.attrs) > 0:
        print("\n📋 文件级属性:")
        for key, value in f.attrs.items():
            print(f"  {key}: {value}")


📂 HDF5 文件结构

顶层键: ['meta', 'time', 'topics', 'valid']

📁 meta: Group
   Attributes: {'created_by': 'convert_ros2bag_to_hdf5_native.py', 'end_time': 1767965094.3115618, 'hdfview_compatible': False, 'scheme': 'B', 'start_time': 1767965088.2484832, 'target_fps': 30.0, 'timeline_length': 183}
📄 time: Dataset
   Shape: (183)
   Dtype: int64
📁 topics: Group
📁 _camera_camera_aligned_depth_to_color_image_raw: Group
   Attributes: {'compressed': True, 'encoding': '16UC1', 'fixed_length_compression': True, 'max_compressed_size': 921600, 'original_height': 480, 'original_width': 640, 'topic': '/camera/camera/aligned_depth_to_color/image_raw'}
📄 data: Dataset
   Shape: (183 x 921600)
   Dtype: uint8
📄 data_length: Dataset
   Shape: (183)
   Dtype: int32
📁 _camera_camera_color_image_raw: Group
   Attributes: {'compressed': True, 'encoding': 'rgb8', 'fixed_length_compression': True, 'max_compressed_size': 921600, 'original_height': 480, 'original_width': 640, 'topic': '/camera/camera/color/image_raw

## 2. 读取所有数据到内存


In [4]:
def load_all_hdf5_data(file_path):
    """加载 HDF5 文件中的所有数据"""
    data = {}
    
    with h5py.File(file_path, 'r') as f:
        # 读取顶层数据
        for key in f.keys():
            if isinstance(f[key], h5py.Dataset):
                data[key] = f[key][:]
            elif isinstance(f[key], h5py.Group):
                data[key] = {}
                load_group(f[key], data[key])
        
        # 读取文件属性
        if hasattr(f, 'attrs') and len(f.attrs) > 0:
            data['_file_attrs'] = dict(f.attrs)
    
    return data

def load_group(group, data_dict):
    """递归加载组中的所有数据"""
    for key in group.keys():
        if isinstance(group[key], h5py.Dataset):
            data_dict[key] = group[key][:]
        elif isinstance(group[key], h5py.Group):
            data_dict[key] = {}
            load_group(group[key], data_dict[key])
        # 加载属性
        if hasattr(group[key], 'attrs') and len(group[key].attrs) > 0:
            data_dict[f"{key}_attrs"] = dict(group[key].attrs)

# 加载所有数据
print("🔄 正在加载所有数据...")
all_data = load_all_hdf5_data(hdf5_path)
print("✅ 数据加载完成！")

# 打印数据摘要
print("\n" + "=" * 80)
print("📊 数据摘要")
print("=" * 80)
for key, value in all_data.items():
    if key == '_file_attrs':
        print(f"\n📋 {key}:")
        for attr_key, attr_value in value.items():
            print(f"  {attr_key}: {attr_value}")
    elif isinstance(value, dict):
        print(f"\n📁 {key}:")
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, np.ndarray):
                print(f"  {sub_key}: shape={sub_value.shape}, dtype={sub_value.dtype}")
            elif isinstance(sub_value, dict):
                print(f"  {sub_key}: (nested dict with {len(sub_value)} keys)")
            else:
                print(f"  {sub_key}: {type(sub_value)}")
    elif isinstance(value, np.ndarray):
        print(f"\n📄 {key}: shape={value.shape}, dtype={value.dtype}")
    else:
        print(f"\n📄 {key}: {type(value)}")


🔄 正在加载所有数据...
✅ 数据加载完成！

📊 数据摘要

📁 meta:

📄 time: shape=(183,), dtype=int64

📁 topics:
  _camera_camera_aligned_depth_to_color_image_raw: (nested dict with 2 keys)
  _camera_camera_aligned_depth_to_color_image_raw_attrs: (nested dict with 7 keys)
  _camera_camera_color_image_raw: (nested dict with 2 keys)
  _camera_camera_color_image_raw_attrs: (nested dict with 7 keys)
  _camera_camera_depth_image_rect_raw: (nested dict with 2 keys)
  _camera_camera_depth_image_rect_raw_attrs: (nested dict with 7 keys)
  _control_Elbow_left: (nested dict with 3 keys)
  _control_Elbow_left_attrs: (nested dict with 3 keys)
  _control_Elbow_right: (nested dict with 3 keys)
  _control_Elbow_right_attrs: (nested dict with 3 keys)
  _control_eef_cmd_A: (nested dict with 3 keys)
  _control_eef_cmd_A_attrs: (nested dict with 3 keys)
  _control_eef_cmd_B: (nested dict with 3 keys)
  _control_eef_cmd_B_attrs: (nested dict with 3 keys)
  _control_gripperValueL: (nested dict with 2 keys)
  _control_gripperValueL_

## 3. 详细查看各个 Topic 的数据


In [ ]:
# 检查 topics 数据
if 'topics' in all_data:
    print("=" * 80)
    print("📡 Topics 数据详情")
    print("=" * 80)
    
    topics = all_data['topics']
    for topic_name in topics.keys():
        print(f"\n📡 Topic: {topic_name}")
        topic_data = topics[topic_name]
        
        if isinstance(topic_data, dict):
            for data_key, data_value in topic_data.items():
                if isinstance(data_value, np.ndarray):
                    print(f"  {data_key}:")
                    print(f"    Shape: {data_value.shape}")
                    print(f"    Dtype: {data_value.dtype}")
                    print(f"    Min: {data_value.min() if data_value.size > 0 else 'N/A'}")
                    print(f"    Max: {data_value.max() if data_value.size > 0 else 'N/A'}")
                    print(f"    Mean: {data_value.mean() if data_value.size > 0 else 'N/A'}")
                    if data_value.size > 0 and data_value.size <= 20:
                        print(f"    Values: {data_value}")
                    elif data_value.size > 0:
                        print(f"    First 5 values: {data_value[:5] if len(data_value.shape) == 1 else data_value[:5]}")
                else:
                    print(f"  {data_key}: {type(data_value)}")
        else:
            print(f"  Type: {type(topic_data)}")
            if isinstance(topic_data, np.ndarray):
                print(f"  Shape: {topic_data.shape}")
                print(f"  Dtype: {topic_data.dtype}")


📡 Topics 数据详情

📡 Topic: _camera_camera_aligned_depth_to_color_image_raw
  data:
    Shape: (183, 921600)
    Dtype: uint8
    Min: 0
    Max: 255
    Mean: 1.8569659679530965
    First 5 values: [[255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]]
  data_length:
    Shape: (183,)
    Dtype: int32
    Min: 0
    Max: 16665
    Mean: 15000.91256830601
    First 5 values: [13968 13945 14532 14784 14650]

📡 Topic: _camera_camera_aligned_depth_to_color_image_raw_attrs
  compressed: <class 'numpy.bool_'>
  encoding: <class 'str'>
  fixed_length_compression: <class 'numpy.bool_'>
  max_compressed_size: <class 'numpy.int64'>
  original_height: <class 'numpy.int64'>
  original_width: <class 'numpy.int64'>
  topic: <class 'str'>

📡 Topic: _camera_camera_color_image_raw
  data:
    Shape: (183, 921600)
    Dtype: uint8
    Min: 0
    Max: 255
    Mean: 2.329246653479812
    First 5 values: [[255

UFuncTypeError: ufunc 'minimum' did not contain a loop with signature matching types (dtype('S2'), dtype('S2')) -> None

## 4. 读取关节状态数据


In [ ]:
# 读取关节状态
if 'topics' in all_data and '_joint_states' in all_data['topics']:
    joint_states = all_data['topics']['_joint_states']
    
    if 'position' in joint_states:
        positions = joint_states['position']
        print("=" * 80)
        print("🦾 关节位置数据")
        print("=" * 80)
        print(f"Shape: {positions.shape}")
        print(f"Dtype: {positions.dtype}")
        print(f"时间步数: {positions.shape[0]}")
        print(f"关节数: {positions.shape[1]}")
        print(f"\n前 5 个时间步的关节位置:")
        print(positions[:5])
        print(f"\n位置统计:")
        print(f"  Min: {positions.min(axis=0)}")
        print(f"  Max: {positions.max(axis=0)}")
        print(f"  Mean: {positions.mean(axis=0)}")
        print(f"  Std: {positions.std(axis=0)}")
    
    if 'velocity' in joint_states:
        velocities = joint_states['velocity']
        print("\n" + "=" * 80)
        print("⚡ 关节速度数据")
        print("=" * 80)
        print(f"Shape: {velocities.shape}")
        print(f"Dtype: {velocities.dtype}")
        print(f"时间步数: {velocities.shape[0]}")
        print(f"关节数: {velocities.shape[1]}")
        print(f"\n前 5 个时间步的关节速度:")
        print(velocities[:5])
        print(f"\n速度统计:")
        print(f"  Min: {velocities.min(axis=0)}")
        print(f"  Max: {velocities.max(axis=0)}")
        print(f"  Mean: {velocities.mean(axis=0)}")
        print(f"  Std: {velocities.std(axis=0)}")
else:
    print("⚠️ 未找到关节状态数据")


🦾 关节位置数据
Shape: (183, 14)
Dtype: float64
时间步数: 183
关节数: 14

前 5 个时间步的关节位置:
[[        nan         nan         nan         nan         nan         nan
          nan         nan         nan         nan         nan         nan
          nan         nan]
 [ 2.44516039 -1.25108433 -1.47270131 -2.36863971 -0.12651795  0.29253373
   0.20946185 -1.90543234 -1.25535023  1.49772227 -2.24315763  0.39921468
   0.63047075 -0.44681731]
 [ 2.44516039 -1.25108421 -1.4727006  -2.36863923 -0.12651758  0.29253361
   0.20946124 -1.90127921 -1.25558352  1.49542463 -2.24090528  0.39869073
   0.63162345 -0.44681266]
 [ 2.44515967 -1.25108421 -1.4727006  -2.36863971 -0.1265171   0.29253387
   0.20946136 -1.89057136 -1.25644267  1.49015379 -2.23562145  0.39772838
   0.63431215 -0.44680306]
 [ 2.44516039 -1.25108433 -1.47270083 -2.36863995 -0.12651663  0.29253456
   0.20946124 -1.87708116 -1.25770724  1.48505616 -2.23059225  0.39675707
   0.63758308 -0.44678149]]

位置统计:
  Min: [nan nan nan nan nan nan nan nan na

## 5. 读取和可视化图像数据


In [ ]:
def decode_image(img_data: np.ndarray, img_length: int) -> np.ndarray:
    """解码图像数据"""
    img_bytes = bytes(img_data[:img_length])
    
    # 尝试作为 JPEG 解码
    try:
        nparr = np.frombuffer(img_bytes, np.uint8)
        img = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
        if img is not None:
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            return img
    except:
        pass
    
    # 尝试用 PIL
    try:
        from io import BytesIO
        img = Image.open(BytesIO(img_bytes))
        if img.mode != 'RGB':
            img = img.convert('RGB')
        return np.array(img)
    except:
        pass
    
    # 尝试作为原始图像数据
    possible_sizes = [
        (720, 1280, 1), (480, 640, 3), (480, 854, 3), (360, 640, 3),
    ]
    for h, w, c in possible_sizes:
        if h * w * c == img_length:
            img = np.frombuffer(img_bytes, dtype=np.uint8).reshape(h, w, c)
            if c == 1:
                img = np.repeat(img, 3, axis=2)
            return img
    
    # 默认尝试 640x480x3
    if img_length >= 640 * 480 * 3:
        img = np.frombuffer(img_bytes[:640*480*3], dtype=np.uint8).reshape(480, 640, 3)
        return img
    
    raise ValueError(f"无法解码图像数据，长度: {img_length}")

# 读取图像数据
if 'topics' in all_data and '_camera_camera_color_image_raw' in all_data['topics']:
    image_topic = all_data['topics']['_camera_camera_color_image_raw']
    
    if 'data' in image_topic and 'data_length' in image_topic:
        image_data = image_topic['data']
        image_lengths = image_topic['data_length']
        
        print("=" * 80)
        print("📷 图像数据")
        print("=" * 80)
        print(f"图像数据 Shape: {image_data.shape}")
        print(f"图像长度数组 Shape: {image_lengths.shape}")
        print(f"图像数量: {len(image_lengths)}")
        print(f"图像长度范围: {image_lengths.min()} - {image_lengths.max()}")
        
        # 解码前几张图像
        num_images_to_decode = min(5, len(image_lengths))
        decoded_images = []
        
        print(f"\n🔄 正在解码前 {num_images_to_decode} 张图像...")
        for i in range(num_images_to_decode):
            try:
                img = decode_image(image_data[i], image_lengths[i])
                decoded_images.append(img)
                print(f"  ✅ 图像 {i}: {img.shape}")
            except Exception as e:
                print(f"  ❌ 图像 {i} 解码失败: {e}")
        
        # 可视化图像
        if len(decoded_images) > 0:
            fig, axes = plt.subplots(1, len(decoded_images), figsize=(5*len(decoded_images), 5))
            if len(decoded_images) == 1:
                axes = [axes]
            for i, img in enumerate(decoded_images):
                axes[i].imshow(img)
                axes[i].set_title(f'图像 {i}')
                axes[i].axis('off')
            plt.tight_layout()
            plt.show()
            
            # 保存解码后的图像到变量
            images = decoded_images
        else:
            images = []
    else:
        print("⚠️ 未找到图像数据或图像长度信息")
        images = []
else:
    print("⚠️ 未找到图像 topic")
    images = []


## 6. 读取有效性标记（如果有）


In [ ]:
# 读取有效性标记
if 'valid' in all_data:
    print("=" * 80)
    print("✅ 有效性标记数据")
    print("=" * 80)
    
    valid_data = all_data['valid']
    for key, value in valid_data.items():
        if isinstance(value, np.ndarray):
            print(f"\n{key}:")
            print(f"  Shape: {value.shape}")
            print(f"  Dtype: {value.dtype}")
            print(f"  有效数据点: {value.sum()} / {len(value)} ({100*value.sum()/len(value):.1f}%)")
            print(f"  无效数据点: {(~value).sum()} / {len(value)} ({100*(~value).sum()/len(value):.1f}%)")
else:
    print("⚠️ 未找到有效性标记数据")


## 7. 读取时间戳数据（如果有）


In [ ]:
# 读取时间戳
if 'time' in all_data:
    time_data = all_data['time']
    print("=" * 80)
    print("⏰ 时间戳数据")
    print("=" * 80)
    print(f"Shape: {time_data.shape}")
    print(f"Dtype: {time_data.dtype}")
    print(f"时间步数: {len(time_data)}")
    print(f"起始时间: {time_data[0]}")
    print(f"结束时间: {time_data[-1]}")
    print(f"总时长: {time_data[-1] - time_data[0]:.6f} 秒")
    
    if len(time_data) > 1:
        dt = np.diff(time_data)
        print(f"\n时间间隔统计:")
        print(f"  平均: {dt.mean():.6f} 秒")
        print(f"  最小: {dt.min():.6f} 秒")
        print(f"  最大: {dt.max():.6f} 秒")
        print(f"  标准差: {dt.std():.6f} 秒")
        
        # 可视化时间间隔
        plt.figure(figsize=(12, 4))
        plt.subplot(1, 2, 1)
        plt.plot(dt)
        plt.xlabel('时间步')
        plt.ylabel('时间间隔 (秒)')
        plt.title('时间间隔序列')
        plt.grid(True)
        
        plt.subplot(1, 2, 2)
        plt.hist(dt, bins=50)
        plt.xlabel('时间间隔 (秒)')
        plt.ylabel('频数')
        plt.title('时间间隔分布')
        plt.grid(True)
        plt.tight_layout()
        plt.show()
else:
    print("⚠️ 未找到时间戳数据")


## 8. 数据统计总结


In [ ]:
print("=" * 80)
print("📊 数据统计总结")
print("=" * 80)

# 统计所有数据
total_size = 0
for key, value in all_data.items():
    if key == '_file_attrs':
        continue
    elif isinstance(value, dict):
        for sub_key, sub_value in value.items():
            if isinstance(sub_value, np.ndarray):
                size_mb = sub_value.nbytes / (1024 * 1024)
                total_size += size_mb
                print(f"  {key}/{sub_key}: {size_mb:.2f} MB")
    elif isinstance(value, np.ndarray):
        size_mb = value.nbytes / (1024 * 1024)
        total_size += size_mb
        print(f"  {key}: {size_mb:.2f} MB")

print(f"\n💾 总数据大小: {total_size:.2f} MB")

# 打印关键数据维度
print("\n📐 关键数据维度:")
if 'topics' in all_data:
    if '_joint_states' in all_data['topics']:
        js = all_data['topics']['_joint_states']
        if 'position' in js:
            print(f"  关节位置: {js['position'].shape}")
        if 'velocity' in js:
            print(f"  关节速度: {js['velocity'].shape}")
    
    if '_camera_camera_color_image_raw' in all_data['topics']:
        img_topic = all_data['topics']['_camera_camera_color_image_raw']
        if 'data' in img_topic:
            print(f"  图像数据: {img_topic['data'].shape}")
        if 'data_length' in img_topic:
            print(f"  图像长度: {img_topic['data_length'].shape}")

if 'time' in all_data:
    print(f"  时间戳: {all_data['time'].shape}")

print("\n✅ 所有数据已加载到 `all_data` 变量中，可以在后续单元格中使用！")


## 9. 可选：保存提取的数据到文件

如果需要，可以将提取的数据保存为 numpy 文件或其他格式。


In [ ]:
# 示例：保存关节位置和速度数据
save_data = False  # 设置为 True 来保存数据

if save_data and 'topics' in all_data and '_joint_states' in all_data['topics']:
    joint_states = all_data['topics']['_joint_states']
    
    output_dir = Path("pick_blue_bottle_extracted")
    output_dir.mkdir(exist_ok=True)
    
    if 'position' in joint_states:
        np.save(output_dir / "joint_positions.npy", joint_states['position'])
        print(f"✅ 已保存关节位置到: {output_dir / 'joint_positions.npy'}")
    
    if 'velocity' in joint_states:
        np.save(output_dir / "joint_velocities.npy", joint_states['velocity'])
        print(f"✅ 已保存关节速度到: {output_dir / 'joint_velocities.npy'}")
    
    print(f"\n💡 提示: 要保存数据，请将上面的 `save_data` 设置为 `True`")
else:
    print("💡 提示: 要保存数据，请将上面的 `save_data` 设置为 `True`")


## 10. 检查是否有 ee_pos 和 ee_ori 数据


In [ ]:
import numpy as np
from scipy.spatial.transform import Rotation as R

def check_ee_pos_ori_in_hdf5(hdf5_path):
    """检查 HDF5 文件中是否有 ee_pos 和 ee_ori"""
    print("=" * 80)
    print(f"检查文件: {hdf5_path}")
    print("=" * 80)
    
    with h5py.File(hdf5_path, "r") as f:
        # 方法1: 检查标准 Libero 格式 (data/demo_X/obs/ee_pos, ee_ori)
        print("\n📂 方法1: 检查标准 Libero 格式 (data/demo_X/obs/ee_pos, ee_ori)")
        has_standard_format = False
        if "data" in f:
            data_group = f["data"]
            demos = [key for key in data_group.keys() if key.startswith("demo_")]
            print(f"  找到 {len(demos)} 个 demo")
            
            for demo_key in demos[:3]:  # 只检查前3个
                demo = data_group[demo_key]
                if "obs" in demo:
                    obs = demo["obs"]
                    if "ee_pos" in obs:
                        ee_pos = obs["ee_pos"]
                        print(f"  ✅ {demo_key}/obs/ee_pos: shape={ee_pos.shape}, dtype={ee_pos.dtype}")
                        has_standard_format = True
                    else:
                        print(f"  ❌ {demo_key}/obs/ee_pos: 不存在")
                    
                    if "ee_ori" in obs:
                        ee_ori = obs["ee_ori"]
                        print(f"  ✅ {demo_key}/obs/ee_ori: shape={ee_ori.shape}, dtype={ee_ori.dtype}")
                        has_standard_format = True
                    else:
                        print(f"  ❌ {demo_key}/obs/ee_ori: 不存在")
                    
                    if "ee_states" in obs:
                        ee_states = obs["ee_states"]
                        print(f"  ✅ {demo_key}/obs/ee_states: shape={ee_states.shape}, dtype={ee_states.dtype}")
                        print(f"     (ee_states 包含 ee_pos 和 ee_ori，前3维是pos，后3维是ori)")
        else:
            print("  ❌ 没有找到 'data' 组（不是标准 Libero 格式）")
        
        # 方法2: 检查 topics 中是否有 eef 相关信息
        print("\n📂 方法2: 检查 topics 中的 eef 数据")
        eef_data_found = {}
        if "topics" in f:
            topics = f["topics"]
            eef_topics = [key for key in topics.keys() if "eef" in key.lower() or "end_effector" in key.lower()]
            
            if eef_topics:
                print(f"  找到 {len(eef_topics)} 个 eef 相关的 topic:")
                for topic_key in eef_topics:
                    topic = topics[topic_key]
                    print(f"    📡 {topic_key}:")
                    if isinstance(topic, h5py.Group):
                        for sub_key in topic.keys():
                            dataset = topic[sub_key]
                            if isinstance(dataset, h5py.Dataset):
                                print(f"      {sub_key}: shape={dataset.shape}, dtype={dataset.dtype}")
                                # 检查是否有 position 和 orientation
                                if sub_key == "position":
                                    print(f"        ✅ 找到 position (可能是 ee_pos)")
                                    eef_data_found[f"{topic_key}/position"] = dataset
                                elif sub_key == "orientation":
                                    print(f"        ✅ 找到 orientation (可能是 ee_ori)")
                                    eef_data_found[f"{topic_key}/orientation"] = dataset
                    elif isinstance(topic, h5py.Dataset):
                        print(f"      Dataset: shape={topic.shape}, dtype={topic.dtype}")
            else:
                print("  ❌ 没有找到 eef 相关的 topic")
        
        # 方法3: 递归搜索所有可能的路径
        print("\n📂 方法3: 递归搜索所有包含 'ee' 或 'eef' 的数据")
        ee_paths = []
        def search_for_ee(name, obj):
            if "ee" in name.lower() or "eef" in name.lower() or "end_effector" in name.lower():
                if isinstance(obj, h5py.Dataset):
                    print(f"  ✅ 找到: {name}")
                    print(f"     shape={obj.shape}, dtype={obj.dtype}")
                    ee_paths.append(name)
                elif isinstance(obj, h5py.Group):
                    print(f"  ✅ 找到组: {name}")
                    for key in obj.keys():
                        if isinstance(obj[key], h5py.Dataset):
                            print(f"     {key}: shape={obj[key].shape}, dtype={obj[key].dtype}")
                            ee_paths.append(f"{name}/{key}")
        
        f.visititems(search_for_ee)
        
        # 总结
        print("\n" + "=" * 80)
        print("📊 检查结果总结")
        print("=" * 80)
        if has_standard_format:
            print("✅ 找到标准 Libero 格式的 ee_pos 和 ee_ori")
        elif eef_data_found:
            print(f"✅ 找到 {len(eef_data_found)} 个可能的 eef 数据源:")
            for key in eef_data_found.keys():
                print(f"  - {key}")
        elif ee_paths:
            print(f"✅ 找到 {len(ee_paths)} 个包含 'ee' 的数据路径:")
            for path in ee_paths[:10]:  # 只显示前10个
                print(f"  - {path}")
        else:
            print("❌ 没有找到 ee_pos 或 ee_ori 数据")
            print("\n💡 建议:")
            print("  1. 检查是否有 joint_states，可以通过正向运动学计算 ee_pos/ee_ori")
            print("  2. 检查是否有控制命令中的目标位置")
        
        return has_standard_format, eef_data_found, ee_paths

# 检查文件
has_ee, eef_data, ee_paths = check_ee_pos_ori_in_hdf5(hdf5_path)


检查文件: pick_blue_bottle/rosbag2_2026_01_09-21_24_48/rosbag2_2026_01_09-21_24_48_0.h5

📂 方法1: 检查标准 Libero 格式 (data/demo_X/obs/ee_pos, ee_ori)
  ❌ 没有找到 'data' 组（不是标准 Libero 格式）

📂 方法2: 检查 topics 中的 eef 数据
  找到 4 个 eef 相关的 topic:
    📡 _control_eef_cmd_A:
      names: shape=(7,), dtype=|S2
      orientation: shape=(183, 4), dtype=float64
        ✅ 找到 orientation (可能是 ee_ori)
      position: shape=(183, 3), dtype=float64
        ✅ 找到 position (可能是 ee_pos)
    📡 _control_eef_cmd_B:
      names: shape=(7,), dtype=|S2
      orientation: shape=(183, 4), dtype=float64
        ✅ 找到 orientation (可能是 ee_ori)
      position: shape=(183, 3), dtype=float64
        ✅ 找到 position (可能是 ee_pos)
    📡 _info_eef_left:
      names: shape=(7,), dtype=|S2
      orientation: shape=(183, 4), dtype=float64
        ✅ 找到 orientation (可能是 ee_ori)
      position: shape=(183, 3), dtype=float64
        ✅ 找到 position (可能是 ee_pos)
    📡 _info_eef_right:
      names: shape=(7,), dtype=|S2
      orientation: shape=(183, 4)

## 11. 读取并显示找到的 ee_pos 和 ee_ori 数据


In [ ]:
# 从 topics 中读取 eef 数据（如果存在）
if 'topics' in all_data:
    topics = all_data['topics']
    
    # 查找 eef 相关的 topics
    eef_topics = [key for key in topics.keys() if 'eef' in key.lower() or 'info' in key.lower()]
    
    print("=" * 80)
    print("📡 从 topics 中读取 eef 数据")
    print("=" * 80)
    
    ee_pos_data = None
    ee_ori_data = None
    
    for topic_key in eef_topics:
        if isinstance(topics[topic_key], dict):
            topic_data = topics[topic_key]
            
            # 检查 _info_eef_left 或 _info_eef_right
            if 'eef' in topic_key.lower():
                print(f"\n📡 Topic: {topic_key}")
                
                if 'position' in topic_data:
                    pos = topic_data['position']
                    print(f"  ✅ position: shape={pos.shape}, dtype={pos.dtype}")
                    print(f"     前5个值:\n{pos[:5]}")
                    if ee_pos_data is None:
                        ee_pos_data = pos
                
                if 'orientation' in topic_data:
                    ori = topic_data['orientation']
                    print(f"  ✅ orientation: shape={ori.shape}, dtype={ori.dtype}")
                    print(f"     前5个值:\n{ori[:5]}")
                    if ee_ori_data is None:
                        ee_ori_data = ori
    
    # 如果找到了数据，显示统计信息
    if ee_pos_data is not None:
        print("\n" + "=" * 80)
        print("📊 ee_pos 统计信息")
        print("=" * 80)
        print(f"Shape: {ee_pos_data.shape}")
        print(f"Dtype: {ee_pos_data.dtype}")
        print(f"Min: {ee_pos_data.min(axis=0)}")
        print(f"Max: {ee_pos_data.max(axis=0)}")
        print(f"Mean: {ee_pos_data.mean(axis=0)}")
        print(f"Std: {ee_pos_data.std(axis=0)}")
    
    if ee_ori_data is not None:
        print("\n" + "=" * 80)
        print("📊 ee_ori 统计信息")
        print("=" * 80)
        print(f"Shape: {ee_ori_data.shape}")
        print(f"Dtype: {ee_ori_data.dtype}")
        print(f"Min: {ee_ori_data.min(axis=0)}")
        print(f"Max: {ee_ori_data.max(axis=0)}")
        print(f"Mean: {ee_ori_data.mean(axis=0)}")
        print(f"Std: {ee_ori_data.std(axis=0)}")
        print("\n💡 注意: orientation 可能是四元数 (x, y, z, w) 格式，需要转换为轴角")
        
        # 检查是否是四元数（4维）还是轴角（3维）
        if ee_ori_data.shape[1] == 4:
            print("  ⚠️  检测到 orientation 是 4 维，可能是四元数格式")
            print("  需要转换为轴角格式才能使用 compute_geom_action")
        elif ee_ori_data.shape[1] == 3:
            print("  ✅ orientation 是 3 维，可能是轴角格式，可以直接使用")
    
    # 总结
    print("\n" + "=" * 80)
    print("📋 数据可用性总结")
    print("=" * 80)
    if ee_pos_data is not None and ee_ori_data is not None:
        print("✅ 找到 ee_pos 和 ee_ori 数据！")
        print(f"   ee_pos 路径: topics/{[k for k in topics.keys() if 'eef' in k.lower()][0]}/position")
        print(f"   ee_ori 路径: topics/{[k for k in topics.keys() if 'eef' in k.lower()][0]}/orientation")
        print(f"\n   可以使用这些数据来计算几何动作 (geometric action)")
    elif ee_pos_data is not None:
        print("⚠️  只找到 ee_pos，缺少 ee_ori")
    elif ee_ori_data is not None:
        print("⚠️  只找到 ee_ori，缺少 ee_pos")
    else:
        print("❌ 没有找到 ee_pos 或 ee_ori 数据")
        print("\n💡 替代方案:")
        print("  1. 从 joint_states 通过正向运动学计算")
        print("  2. 从控制命令中的目标位置提取")
else:
    print("❌ 没有找到 'topics' 组")


📡 从 topics 中读取 eef 数据

📡 Topic: _control_eef_cmd_A
  ✅ position: shape=(183, 3), dtype=float64
     前5个值:
[[       nan        nan        nan]
 [0.165      0.18552534 0.1688796 ]
 [0.165      0.18609523 0.17003854]
 [0.165      0.18672449 0.17080854]
 [0.165      0.18760958 0.17125537]]
  ✅ orientation: shape=(183, 4), dtype=float64
     前5个值:
[[        nan         nan         nan         nan]
 [-0.53969136  0.38821489 -0.4133455   0.62222826]
 [-0.53965809  0.38668115 -0.41469871  0.62231174]
 [-0.53952719  0.38521217 -0.41469937  0.62333503]
 [-0.53970119  0.38466819 -0.41474414  0.62349055]]

📡 Topic: _control_eef_cmd_A_attrs

📡 Topic: _control_eef_cmd_B
  ✅ position: shape=(183, 3), dtype=float64
     前5个值:
[[        nan         nan         nan]
 [ 0.33047189 -0.21914914  0.18778101]
 [ 0.33196979 -0.21989838  0.18690639]
 [ 0.33298244 -0.22049623  0.18603447]
 [ 0.33412288 -0.22185035  0.18497011]]
  ✅ orientation: shape=(183, 4), dtype=float64
     前5个值:
[[       nan        nan   

## 12. 测试计算几何动作（如果数据可用）


In [8]:
# 如果找到了 ee_pos 和 ee_ori，测试计算几何动作
if 'ee_pos_data' in locals() and 'ee_ori_data' in locals() and ee_pos_data is not None and ee_ori_data is not None:
    # 定义函数（从用户提供的代码）
    def axisangle_to_quat(axis_angle):
        """axis_angle: (3,), return quat: (4,) in xyzw"""
        return R.from_rotvec(axis_angle).as_quat()
    
    def quat_to_axisangle(quat):
        """quat: (4,) xyzw, return axis-angle: (3,)"""
        return R.from_quat(quat).as_rotvec()
    
    def relative_axisangle(aa_t, aa_t1):
        """Compute relative rotation from t -> t+1 in axis-angle"""
        q_t = axisangle_to_quat(aa_t)
        q_t1 = axisangle_to_quat(aa_t1)
        q_rel = R.from_quat(q_t1) * R.from_quat(q_t).inv()
        return q_rel.as_rotvec()
    
    def compute_geom_action(ee_pos_t, ee_ori_t, ee_pos_t1, ee_ori_t1):
        """All inputs are np.ndarray with shape (3,), ee_ori_* are axis-angle"""
        delta_pos = ee_pos_t1 - ee_pos_t
        delta_ori = relative_axisangle(ee_ori_t, ee_ori_t1)
        action_6d = np.concatenate([delta_pos, delta_ori])
        return action_6d
    
    # 检查 orientation 格式
    if ee_ori_data.shape[1] == 4:
        print("⚠️  orientation 是四元数格式，需要转换为轴角")
        print("正在转换...")
        # 转换为轴角
        ee_ori_axisangle = np.array([quat_to_axisangle(ori) for ori in ee_ori_data])
        print(f"✅ 转换完成: shape={ee_ori_axisangle.shape}")
    elif ee_ori_data.shape[1] == 3:
        print("✅ orientation 已经是轴角格式")
        ee_ori_axisangle = ee_ori_data
    else:
        print(f"❌ 未知的 orientation 维度: {ee_ori_data.shape[1]}")
        ee_ori_axisangle = None
    
    if ee_ori_axisangle is not None:
        print("\n" + "=" * 80)
        print("🧪 测试计算几何动作")
        print("=" * 80)
        
        T = min(len(ee_pos_data), len(ee_ori_axisangle))
        print(f"数据长度: {T}")
        
        # 计算前几个时间步的几何动作
        num_test = min(5, T - 1)
        print(f"\n计算前 {num_test} 个时间步的几何动作:\n")
        
        for t in range(num_test):
            action_geom = compute_geom_action(
                ee_pos_data[t],
                ee_ori_axisangle[t],
                ee_pos_data[t + 1],
                ee_ori_axisangle[t + 1],
            )
            
            print(f"t={t:03d}")
            print(f"  ee_pos[{t}]:     {ee_pos_data[t]}")
            print(f"  ee_pos[{t+1}]:   {ee_pos_data[t+1]}")
            print(f"  ee_ori[{t}]:    {ee_ori_axisangle[t]}")
            print(f"  ee_ori[{t+1}]:  {ee_ori_axisangle[t+1]}")
            print(f"  geom_action (Δpos, Δori): {action_geom}")
            print()
        
        print("✅ 几何动作计算成功！")
        print("\n💡 提示: 如果 HDF5 中有 actions 数据，可以对比验证")
        
        # 如果 all_data 中有 actions，进行对比
        if 'topics' in all_data and '_joint_states' in all_data['topics']:
            print("\n📊 注意: 当前 HDF5 中的 actions 可能是关节速度，不是几何动作")
            print("   需要从标准 Libero HDF5 格式中读取 actions 进行对比")
    else:
        print("❌ 无法计算几何动作：缺少有效的 orientation 数据")
else:
    print("⚠️  无法测试：需要先运行前面的单元格找到 ee_pos 和 ee_ori 数据")


⚠️  orientation 是四元数格式，需要转换为轴角
正在转换...


ValueError: Found zero norm quaternions in `quat`.

## 13. 详细查看 _info_eef_left 和 _info_eef_right 的数据结构


In [ ]:
# 详细查看 _info_eef_left 和 _info_eef_right 的数据
print("=" * 80)
print("📡 详细查看 _info_eef_left 和 _info_eef_right")
print("=" * 80)

if 'topics' in all_data:
    topics = all_data['topics']
    
    for eef_topic_name in ['_info_eef_left', '_info_eef_right']:
        if eef_topic_name in topics:
            print(f"\n{'='*80}")
            print(f"📡 Topic: {eef_topic_name}")
            print(f"{'='*80}")
            
            eef_topic = topics[eef_topic_name]
            
            if isinstance(eef_topic, dict):
                print(f"\n包含的键: {list(eef_topic.keys())}")
                
                for key, value in eef_topic.items():
                    if isinstance(value, np.ndarray):
                        print(f"\n  📄 {key}:")
                        print(f"    Shape: {value.shape}")
                        print(f"    Dtype: {value.dtype}")
                        print(f"    时间步数: {value.shape[0]}")
                        
                        if len(value.shape) == 2:
                            print(f"    维度数: {value.shape[1]}")
                            print(f"    Min: {value.min(axis=0)}")
                            print(f"    Max: {value.max(axis=0)}")
                            print(f"    Mean: {value.mean(axis=0)}")
                            print(f"    Std: {value.std(axis=0)}")
                            
                            # 显示前几个时间步的数据
                            print(f"\n    前 5 个时间步的数据:")
                            for i in range(min(5, len(value))):
                                print(f"      t={i:03d}: {value[i]}")
                            
                            # 如果是 orientation，检查是否是四元数
                            if key == 'orientation' and value.shape[1] == 4:
                                print(f"\n    💡 这是四元数格式 (x, y, z, w)")
                                print(f"    需要转换为轴角格式才能用于计算几何动作")
                                
                                # 显示四元数范数（应该是1）
                                quat_norms = np.linalg.norm(value, axis=1)
                                print(f"    四元数范数范围: [{quat_norms.min():.6f}, {quat_norms.max():.6f}]")
                                if np.allclose(quat_norms, 1.0, atol=1e-3):
                                    print(f"    ✅ 四元数已归一化")
                                else:
                                    print(f"    ⚠️  四元数未完全归一化")
                            
                            elif key == 'position' and value.shape[1] == 3:
                                print(f"\n    💡 这是位置数据 (x, y, z)")
                            
                            elif key == 'names':
                                print(f"\n    名称: {value}")
                        else:
                            print(f"    前 5 个值: {value[:5]}")
                    else:
                        print(f"\n  {key}: {type(value)}")
            else:
                print(f"  Type: {type(eef_topic)}")
                if isinstance(eef_topic, np.ndarray):
                    print(f"  Shape: {eef_topic.shape}")
                    print(f"  Dtype: {eef_topic.dtype}")
        
        # 检查是否有属性
        with h5py.File(hdf5_path, 'r') as f:
            if eef_topic_name in f['topics']:
                eef_group = f['topics'][eef_topic_name]
                if hasattr(eef_group, 'attrs') and len(eef_group.attrs) > 0:
                    print(f"\n  📋 属性:")
                    for attr_key, attr_value in eef_group.attrs.items():
                        print(f"    {attr_key}: {attr_value}")
else:
    print("❌ 没有找到 'topics' 组")


## 14. 对比 _info_eef_left 和 _info_eef_right 的数据


In [9]:
# 直接查看 _info_eef_left 和 _info_eef_right 下的数据
print("=" * 80)
print("📡 直接查看 _info_eef_left 和 _info_eef_right 的数据结构")
print("=" * 80)

with h5py.File(hdf5_path, 'r') as f:
    for topic_name in ['_info_eef_left', '_info_eef_right']:
        print(f"\n{'='*80}")
        print(f"📡 Topic: {topic_name}")
        print(f"{'='*80}")
        
        if topic_name in f['topics']:
            topic_group = f['topics'][topic_name]
            
            # 显示属性
            if hasattr(topic_group, 'attrs') and len(topic_group.attrs) > 0:
                print("\n📋 Topic 属性:")
                for attr_key, attr_value in topic_group.attrs.items():
                    print(f"  {attr_key}: {attr_value}")
            
            # 显示所有子键
            print(f"\n📂 包含的数据集/组:")
            for key in topic_group.keys():
                dataset = topic_group[key]
                if isinstance(dataset, h5py.Dataset):
                    print(f"\n  📄 {key}:")
                    print(f"    Type: Dataset")
                    print(f"    Shape: {dataset.shape}")
                    print(f"    Dtype: {dataset.dtype}")
                    
                    # 读取并显示数据
                    data = dataset[:]
                    print(f"    数据形状: {data.shape}")
                    
                    if len(data.shape) == 1:
                        print(f"    前 10 个值: {data[:10]}")
                        print(f"    Min: {data.min()}, Max: {data.max()}, Mean: {data.mean():.6f}")
                    elif len(data.shape) == 2:
                        print(f"    前 5 行数据:")
                        for i in range(min(5, len(data))):
                            print(f"      [{i}]: {data[i]}")
                        print(f"    Min (每列): {data.min(axis=0)}")
                        print(f"    Max (每列): {data.max(axis=0)}")
                        print(f"    Mean (每列): {data.mean(axis=0)}")
                    
                    # 特殊处理
                    if key == 'position':
                        print(f"    💡 这是末端执行器位置 (x, y, z)")
                    elif key == 'orientation':
                        if data.shape[-1] == 4:
                            print(f"    💡 这是末端执行器方向四元数 (x, y, z, w)")
                            # 检查四元数范数
                            if len(data.shape) == 2:
                                norms = np.linalg.norm(data, axis=1)
                                print(f"    四元数范数范围: [{norms.min():.6f}, {norms.max():.6f}]")
                        elif data.shape[-1] == 3:
                            print(f"    💡 这是末端执行器方向轴角 (rx, ry, rz)")
                    elif key == 'names':
                        print(f"    💡 字段名称: {data}")
                
                elif isinstance(dataset, h5py.Group):
                    print(f"\n  📁 {key}: Group")
                    print(f"    包含: {list(dataset.keys())}")
        else:
            print(f"  ❌ 未找到 {topic_name}")

print("\n" + "=" * 80)
print("📊 总结")
print("=" * 80)
print("这两个 topic 通常包含:")
print("  - position: (T, 3) 末端执行器位置 [x, y, z]")
print("  - orientation: (T, 4) 末端执行器方向四元数 [x, y, z, w]")
print("  - names: (7,) 字段名称")
print("\n💡 可以使用这些数据来计算几何动作 (geometric action)")


📡 直接查看 _info_eef_left 和 _info_eef_right 的数据结构

📡 Topic: _info_eef_left

📋 Topic 属性:
  dof: 7
  topic: /info/eef_left
  type: Pose

📂 包含的数据集/组:

  📄 names:
    Type: Dataset
    Shape: (7,)
    Dtype: |S2
    数据形状: (7,)
    前 10 个值: [b'x' b'y' b'z' b'qx' b'qy' b'qz' b'qw']


UFuncTypeError: ufunc 'minimum' did not contain a loop with signature matching types (dtype('S2'), dtype('S2')) -> None

## 16. 保存 _joint_states 的全部数据到 txt 文件


In [ ]:
# 保存 _joint_states 的全部数据到 txt 文件
print("=" * 80)
print("💾 保存 _joint_states 数据到 txt 文件")
print("=" * 80)

output_file = Path("pick_blue_bottle_joint_states.txt")

with h5py.File(hdf5_path, 'r') as f:
    if '_joint_states' in f['topics']:
        joint_states_group = f['topics']['_joint_states']
        
        print(f"\n📂 _joint_states 包含的数据:")
        for key in joint_states_group.keys():
            dataset = joint_states_group[key]
            if isinstance(dataset, h5py.Dataset):
                print(f"  - {key}: shape={dataset.shape}, dtype={dataset.dtype}")
        
        # 读取所有数据
        print("\n🔄 正在读取数据...")
        joint_states_data = {}
        for key in joint_states_group.keys():
            dataset = joint_states_group[key]
            if isinstance(dataset, h5py.Dataset):
                joint_states_data[key] = dataset[:]
                print(f"  ✅ 已读取 {key}: {joint_states_data[key].shape}")
        
        # 读取属性
        attrs = {}
        if hasattr(joint_states_group, 'attrs') and len(joint_states_group.attrs) > 0:
            attrs = dict(joint_states_group.attrs)
            print(f"\n📋 属性: {attrs}")
        
        # 保存到 txt 文件
        print(f"\n💾 正在保存到: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as txt_file:
            # 写入头部信息
            txt_file.write("=" * 80 + "\n")
            txt_file.write("Joint States 数据\n")
            txt_file.write("=" * 80 + "\n\n")
            txt_file.write(f"文件来源: {hdf5_path}\n")
            txt_file.write(f"保存时间: {np.datetime64('now')}\n\n")
            
            # 写入属性
            if attrs:
                txt_file.write("属性信息:\n")
                for attr_key, attr_value in attrs.items():
                    txt_file.write(f"  {attr_key}: {attr_value}\n")
                txt_file.write("\n")
            
            # 写入每个数据集
            for key, data in joint_states_data.items():
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"数据集: {key}\n")
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"Shape: {data.shape}\n")
                txt_file.write(f"Dtype: {data.dtype}\n")
                
                if len(data.shape) == 1:
                    txt_file.write(f"\n数据 (一维数组，共 {len(data)} 个元素):\n")
                    for i, val in enumerate(data):
                        txt_file.write(f"  [{i:4d}]: {val}\n")
                elif len(data.shape) == 2:
                    txt_file.write(f"\n数据 (二维数组，{data.shape[0]} 行 x {data.shape[1]} 列):\n")
                    txt_file.write(f"  列索引: ")
                    if 'names' in joint_states_data:
                        names = joint_states_data['names']
                        if len(names) == data.shape[1]:
                            txt_file.write(" | ".join([f"{i:2d}:{name.decode() if isinstance(name, bytes) else name}" 
                                                      for i, name in enumerate(names)]) + "\n")
                        else:
                            txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    else:
                        txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    txt_file.write("  " + "-" * 80 + "\n")
                    
                    for i in range(len(data)):
                        row_str = " | ".join([f"{val:12.6f}" for val in data[i]])
                        txt_file.write(f"  [{i:4d}]: {row_str}\n")
                else:
                    txt_file.write(f"\n数据 (多维数组):\n")
                    txt_file.write(f"  {data}\n")
                
                txt_file.write("\n")
                
                # 写入统计信息（仅对数值类型数据）
                if len(data.shape) >= 1 and data.size > 0:
                    # 检查是否为数值类型（排除字符串类型）
                    is_numeric = np.issubdtype(data.dtype, np.number)
                    if is_numeric:
                        txt_file.write("统计信息:\n")
                        if len(data.shape) == 1:
                            txt_file.write(f"  Min: {data.min()}\n")
                            txt_file.write(f"  Max: {data.max()}\n")
                            txt_file.write(f"  Mean: {data.mean()}\n")
                            txt_file.write(f"  Std: {data.std()}\n")
                        elif len(data.shape) == 2:
                            txt_file.write(f"  Min (每列): {data.min(axis=0)}\n")
                            txt_file.write(f"  Max (每列): {data.max(axis=0)}\n")
                            txt_file.write(f"  Mean (每列): {data.mean(axis=0)}\n")
                            txt_file.write(f"  Std (每列): {data.std(axis=0)}\n")
                        txt_file.write("\n")
        
        file_size = output_file.stat().st_size / 1024  # KB
        print(f"✅ 保存完成！")
        print(f"   文件: {output_file}")
        print(f"   大小: {file_size:.2f} KB")
        print(f"   包含 {len(joint_states_data)} 个数据集")
    else:
        print("❌ 未找到 _joint_states")


💾 保存 _joint_states 数据到 txt 文件

📂 _joint_states 包含的数据:
  - effort: shape=(183, 14), dtype=float64
  - names: shape=(14,), dtype=|S8
  - position: shape=(183, 14), dtype=float64
  - velocity: shape=(183, 14), dtype=float64

🔄 正在读取数据...
  ✅ 已读取 effort: (183, 14)
  ✅ 已读取 names: (14,)
  ✅ 已读取 position: (183, 14)
  ✅ 已读取 velocity: (183, 14)

📋 属性: {'dof': 14, 'topic': '/joint_states', 'type': 'JointState'}

💾 正在保存到: pick_blue_bottle_joint_states.txt


UFuncTypeError: ufunc 'minimum' did not contain a loop with signature matching types (dtype('S8'), dtype('S8')) -> None

## 17. 统计 HDF5 文件中的图片数量


In [ ]:
# 统计 HDF5 文件中的图片数量
print("=" * 80)
print("📷 统计 HDF5 文件中的图片数量")
print("=" * 80)

with h5py.File(hdf5_path, 'r') as f:
    if 'topics' in f:
        topics = f['topics']
        
        # 查找所有包含图像数据的 topic
        image_topics = []
        for topic_name in topics.keys():
            topic = topics[topic_name]
            if isinstance(topic, h5py.Group):
                # 检查是否有 data 和 data_length（图像数据的特征）
                if 'data' in topic and 'data_length' in topic:
                    image_topics.append(topic_name)
        
        print(f"\n找到 {len(image_topics)} 个包含图像数据的 topic:\n")
        
        total_images = 0
        for topic_name in image_topics:
            topic = topics[topic_name]
            data_length = topic['data_length'][:]
            
            # 统计有效图像数量（data_length > 0 表示有图像数据）
            valid_images = np.sum(data_length > 0)
            total_images_in_topic = len(data_length)
            
            print(f"📡 {topic_name}:")
            print(f"   总时间步数: {total_images_in_topic}")
            print(f"   有效图像数: {valid_images}")
            print(f"   无效图像数: {total_images_in_topic - valid_images}")
            
            # 显示图像属性（如果有）
            if hasattr(topic, 'attrs'):
                attrs = dict(topic.attrs)
                if 'original_height' in attrs and 'original_width' in attrs:
                    print(f"   图像尺寸: {attrs['original_width']} x {attrs['original_height']}")
                if 'encoding' in attrs:
                    print(f"   编码格式: {attrs['encoding']}")
                if 'compressed' in attrs:
                    print(f"   是否压缩: {attrs['compressed']}")
            
            # 显示前几个图像的长度信息
            if len(data_length) > 0:
                print(f"   图像数据长度范围: {data_length.min()} - {data_length.max()} bytes")
                print(f"   平均图像大小: {data_length[data_length > 0].mean():.0f} bytes" if valid_images > 0 else "   平均图像大小: N/A")
            
            print()
            total_images += valid_images
        
        print("=" * 80)
        print("📊 总结")
        print("=" * 80)
        print(f"总共有 {len(image_topics)} 个图像 topic")
        print(f"总有效图像数: {total_images}")
        
        # 如果只有一个主要的图像 topic，显示详细信息
        color_image_topics = [t for t in image_topics if 'color' in t.lower() and 'image' in t.lower()]
        if color_image_topics:
            main_topic = color_image_topics[0]
            print(f"\n主要图像 topic: {main_topic}")
            main_topic_group = topics[main_topic]
            data_length = main_topic_group['data_length'][:]
            valid_count = np.sum(data_length > 0)
            print(f"  有效图像: {valid_count} 张")
            print(f"  无效图像: {len(data_length) - valid_count} 张")
    else:
        print("❌ 没有找到 'topics' 组")


📷 统计 HDF5 文件中的图片数量

找到 3 个包含图像数据的 topic:

📡 _camera_camera_aligned_depth_to_color_image_raw:
   总时间步数: 183
   有效图像数: 182
   无效图像数: 1
   图像尺寸: 640 x 480
   编码格式: 16UC1
   是否压缩: True
   图像数据长度范围: 0 - 16665 bytes
   平均图像大小: 15083 bytes

📡 _camera_camera_color_image_raw:
   总时间步数: 183
   有效图像数: 182
   无效图像数: 1
   图像尺寸: 640 x 480
   编码格式: rgb8
   是否压缩: True
   图像数据长度范围: 0 - 18869 bytes
   平均图像大小: 18323 bytes

📡 _camera_camera_depth_image_rect_raw:
   总时间步数: 183
   有效图像数: 182
   无效图像数: 1
   图像尺寸: 640 x 480
   编码格式: 16UC1
   是否压缩: True
   图像数据长度范围: 0 - 26627 bytes
   平均图像大小: 19615 bytes

📊 总结
总共有 3 个图像 topic
总有效图像数: 546

主要图像 topic: _camera_camera_aligned_depth_to_color_image_raw
  有效图像: 182 张
  无效图像: 1 张


## 18. 保存 _control_target_poseR 的全部数据到 txt 文件


In [ ]:
# 保存 _control_target_poseR 的全部数据到 txt 文件
print("=" * 80)
print("💾 保存 _control_target_poseR 数据到 txt 文件")
print("=" * 80)

output_file = Path("pick_blue_bottle_control_target_poseR.txt")

with h5py.File(hdf5_path, 'r') as f:
    if 'topics' in f and '_control_target_poseR' in f['topics']:
        poseR_group = f['topics']['_control_target_poseR']
        
        print(f"\n📂 _control_target_poseR 包含的数据:")
        for key in poseR_group.keys():
            dataset = poseR_group[key]
            if isinstance(dataset, h5py.Dataset):
                print(f"  - {key}: shape={dataset.shape}, dtype={dataset.dtype}")
        
        # 读取所有数据
        print("\n🔄 正在读取数据...")
        poseR_data = {}
        for key in poseR_group.keys():
            dataset = poseR_group[key]
            if isinstance(dataset, h5py.Dataset):
                poseR_data[key] = dataset[:]
                print(f"  ✅ 已读取 {key}: {poseR_data[key].shape}")
        
        # 读取属性
        attrs = {}
        if hasattr(poseR_group, 'attrs') and len(poseR_group.attrs) > 0:
            attrs = dict(poseR_group.attrs)
            print(f"\n📋 属性: {attrs}")
        
        # 保存到 txt 文件
        print(f"\n💾 正在保存到: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as txt_file:
            # 写入头部信息
            txt_file.write("=" * 80 + "\n")
            txt_file.write("Control Target PoseR 数据\n")
            txt_file.write("=" * 80 + "\n\n")
            txt_file.write(f"文件来源: {hdf5_path}\n")
            txt_file.write(f"保存时间: {np.datetime64('now')}\n\n")
            
            # 写入属性
            if attrs:
                txt_file.write("属性信息:\n")
                for attr_key, attr_value in attrs.items():
                    txt_file.write(f"  {attr_key}: {attr_value}\n")
                txt_file.write("\n")
            
            # 写入每个数据集
            for key, data in poseR_data.items():
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"数据集: {key}\n")
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"Shape: {data.shape}\n")
                txt_file.write(f"Dtype: {data.dtype}\n")
                
                if len(data.shape) == 1:
                    txt_file.write(f"\n数据 (一维数组，共 {len(data)} 个元素):\n")
                    for i, val in enumerate(data):
                        txt_file.write(f"  [{i:4d}]: {val}\n")
                elif len(data.shape) == 2:
                    txt_file.write(f"\n数据 (二维数组，{data.shape[0]} 行 x {data.shape[1]} 列):\n")
                    txt_file.write(f"  列索引: ")
                    if 'names' in poseR_data:
                        names = poseR_data['names']
                        if len(names) == data.shape[1]:
                            txt_file.write(" | ".join([f"{i:2d}:{name.decode() if isinstance(name, bytes) else name}" 
                                                      for i, name in enumerate(names)]) + "\n")
                        else:
                            txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    else:
                        txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    txt_file.write("  " + "-" * 80 + "\n")
                    
                    for i in range(len(data)):
                        row_str = " | ".join([f"{val:12.6f}" for val in data[i]])
                        txt_file.write(f"  [{i:4d}]: {row_str}\n")
                else:
                    txt_file.write(f"\n数据 (多维数组):\n")
                    txt_file.write(f"  {data}\n")
                
                txt_file.write("\n")
                
                # 写入统计信息（仅对数值类型数据）
                if len(data.shape) >= 1 and data.size > 0:
                    # 检查是否为数值类型（排除字符串类型）
                    is_numeric = np.issubdtype(data.dtype, np.number)
                    if is_numeric:
                        txt_file.write("统计信息:\n")
                        if len(data.shape) == 1:
                            txt_file.write(f"  Min: {data.min()}\n")
                            txt_file.write(f"  Max: {data.max()}\n")
                            txt_file.write(f"  Mean: {data.mean()}\n")
                            txt_file.write(f"  Std: {data.std()}\n")
                        elif len(data.shape) == 2:
                            txt_file.write(f"  Min (每列): {data.min(axis=0)}\n")
                            txt_file.write(f"  Max (每列): {data.max(axis=0)}\n")
                            txt_file.write(f"  Mean (每列): {data.mean(axis=0)}\n")
                            txt_file.write(f"  Std (每列): {data.std(axis=0)}\n")
                        txt_file.write("\n")
        
        file_size = output_file.stat().st_size / 1024  # KB
        print(f"✅ 保存完成！")
        print(f"   文件: {output_file}")
        print(f"   大小: {file_size:.2f} KB")
        print(f"   包含 {len(poseR_data)} 个数据集")
    else:
        print("❌ 未找到 _control_target_poseR")


💾 保存 _control_target_poseR 数据到 txt 文件

📂 _control_target_poseR 包含的数据:
  - names: shape=(7,), dtype=|S2
  - orientation: shape=(183, 4), dtype=float64
  - position: shape=(183, 3), dtype=float64

🔄 正在读取数据...
  ✅ 已读取 names: (7,)
  ✅ 已读取 orientation: (183, 4)
  ✅ 已读取 position: (183, 3)

📋 属性: {'dof': 7, 'topic': '/control/target_poseR', 'type': 'Pose'}

💾 正在保存到: pick_blue_bottle_control_target_poseR.txt


UFuncTypeError: ufunc 'minimum' did not contain a loop with signature matching types (dtype('S2'), dtype('S2')) -> None

## 19. 保存 _control_gripperValueR 的全部数据到 txt 文件


In [ ]:
# 保存 _control_gripperValueR 的全部数据到 txt 文件
print("=" * 80)
print("💾 保存 _control_gripperValueR 数据到 txt 文件")
print("=" * 80)

output_file = Path("pick_blue_bottle_control_gripperValueR.txt")

with h5py.File(hdf5_path, 'r') as f:
    if 'topics' in f and '_control_gripperValueR' in f['topics']:
        gripper_group = f['topics']['_control_gripperValueR']
        
        print(f"\n📂 _control_gripperValueR 包含的数据:")
        for key in gripper_group.keys():
            dataset = gripper_group[key]
            if isinstance(dataset, h5py.Dataset):
                print(f"  - {key}: shape={dataset.shape}, dtype={dataset.dtype}")
        
        # 读取所有数据
        print("\n🔄 正在读取数据...")
        gripper_data = {}
        for key in gripper_group.keys():
            dataset = gripper_group[key]
            if isinstance(dataset, h5py.Dataset):
                gripper_data[key] = dataset[:]
                print(f"  ✅ 已读取 {key}: {gripper_data[key].shape}")
        
        # 读取属性
        attrs = {}
        if hasattr(gripper_group, 'attrs') and len(gripper_group.attrs) > 0:
            attrs = dict(gripper_group.attrs)
            print(f"\n📋 属性: {attrs}")
        
        # 保存到 txt 文件
        print(f"\n💾 正在保存到: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as txt_file:
            # 写入头部信息
            txt_file.write("=" * 80 + "\n")
            txt_file.write("Control GripperValueR 数据\n")
            txt_file.write("=" * 80 + "\n\n")
            txt_file.write(f"文件来源: {hdf5_path}\n")
            txt_file.write(f"保存时间: {np.datetime64('now')}\n\n")
            
            # 写入属性
            if attrs:
                txt_file.write("属性信息:\n")
                for attr_key, attr_value in attrs.items():
                    txt_file.write(f"  {attr_key}: {attr_value}\n")
                txt_file.write("\n")
            
            # 写入每个数据集
            for key, data in gripper_data.items():
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"数据集: {key}\n")
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"Shape: {data.shape}\n")
                txt_file.write(f"Dtype: {data.dtype}\n")
                
                if len(data.shape) == 1:
                    txt_file.write(f"\n数据 (一维数组，共 {len(data)} 个元素):\n")
                    for i, val in enumerate(data):
                        txt_file.write(f"  [{i:4d}]: {val}\n")
                elif len(data.shape) == 2:
                    txt_file.write(f"\n数据 (二维数组，{data.shape[0]} 行 x {data.shape[1]} 列):\n")
                    txt_file.write(f"  列索引: ")
                    if 'names' in gripper_data:
                        names = gripper_data['names']
                        if len(names) == data.shape[1]:
                            txt_file.write(" | ".join([f"{i:2d}:{name.decode() if isinstance(name, bytes) else name}" 
                                                      for i, name in enumerate(names)]) + "\n")
                        else:
                            txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    else:
                        txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    txt_file.write("  " + "-" * 80 + "\n")
                    
                    for i in range(len(data)):
                        row_str = " | ".join([f"{val:12.6f}" for val in data[i]])
                        txt_file.write(f"  [{i:4d}]: {row_str}\n")
                else:
                    txt_file.write(f"\n数据 (多维数组):\n")
                    txt_file.write(f"  {data}\n")
                
                txt_file.write("\n")
                
                # 写入统计信息
                if len(data.shape) >= 1 and data.size > 0:
                    txt_file.write("统计信息:\n")
                    if len(data.shape) == 1:
                        txt_file.write(f"  Min: {data.min()}\n")
                        txt_file.write(f"  Max: {data.max()}\n")
                        txt_file.write(f"  Mean: {data.mean()}\n")
                        txt_file.write(f"  Std: {data.std()}\n")
                    elif len(data.shape) == 2:
                        txt_file.write(f"  Min (每列): {data.min(axis=0)}\n")
                        txt_file.write(f"  Max (每列): {data.max(axis=0)}\n")
                        txt_file.write(f"  Mean (每列): {data.mean(axis=0)}\n")
                        txt_file.write(f"  Std (每列): {data.std(axis=0)}\n")
                    txt_file.write("\n")
        
        file_size = output_file.stat().st_size / 1024  # KB
        print(f"✅ 保存完成！")
        print(f"   文件: {output_file}")
        print(f"   大小: {file_size:.2f} KB")
        print(f"   包含 {len(gripper_data)} 个数据集")
    else:
        print("❌ 未找到 _control_gripperValueR")


💾 保存 _control_gripperValueR 数据到 txt 文件

📂 _control_gripperValueR 包含的数据:
  - data: shape=(183,), dtype=float32
  - names: shape=(1,), dtype=|S13

🔄 正在读取数据...
  ✅ 已读取 data: (183,)
  ✅ 已读取 names: (1,)

📋 属性: {'dof': 1, 'topic': '/control/gripperValueR', 'type': 'Float32'}

💾 正在保存到: pick_blue_bottle_control_gripperValueR.txt


UFuncTypeError: ufunc 'minimum' did not contain a loop with signature matching types (dtype('S13'), dtype('S13')) -> None

## 20. 保存 _info_eef_right 的全部数据到 txt 文件


In [ ]:
# 保存 _info_eef_right 的全部数据到 txt 文件
print("=" * 80)
print("💾 保存 _info_eef_right 数据到 txt 文件")
print("=" * 80)

output_file = Path("pick_blue_bottle_info_eef_right.txt")

with h5py.File(hdf5_path, 'r') as f:
    if 'topics' in f and '_info_eef_right' in f['topics']:
        eef_right_group = f['topics']['_info_eef_right']
        
        print(f"\n📂 _info_eef_right 包含的数据:")
        for key in eef_right_group.keys():
            dataset = eef_right_group[key]
            if isinstance(dataset, h5py.Dataset):
                print(f"  - {key}: shape={dataset.shape}, dtype={dataset.dtype}")
        
        # 读取所有数据
        print("\n🔄 正在读取数据...")
        eef_right_data = {}
        for key in eef_right_group.keys():
            dataset = eef_right_group[key]
            if isinstance(dataset, h5py.Dataset):
                eef_right_data[key] = dataset[:]
                print(f"  ✅ 已读取 {key}: {eef_right_data[key].shape}")
        
        # 读取属性
        attrs = {}
        if hasattr(eef_right_group, 'attrs') and len(eef_right_group.attrs) > 0:
            attrs = dict(eef_right_group.attrs)
            print(f"\n📋 属性: {attrs}")
        
        # 保存到 txt 文件
        print(f"\n💾 正在保存到: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as txt_file:
            # 写入头部信息
            txt_file.write("=" * 80 + "\n")
            txt_file.write("Info EEF Right 数据\n")
            txt_file.write("=" * 80 + "\n\n")
            txt_file.write(f"文件来源: {hdf5_path}\n")
            txt_file.write(f"保存时间: {np.datetime64('now')}\n\n")
            
            # 写入属性
            if attrs:
                txt_file.write("属性信息:\n")
                for attr_key, attr_value in attrs.items():
                    txt_file.write(f"  {attr_key}: {attr_value}\n")
                txt_file.write("\n")
            
            # 写入每个数据集
            for key, data in eef_right_data.items():
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"数据集: {key}\n")
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"Shape: {data.shape}\n")
                txt_file.write(f"Dtype: {data.dtype}\n")
                
                if len(data.shape) == 1:
                    txt_file.write(f"\n数据 (一维数组，共 {len(data)} 个元素):\n")
                    for i, val in enumerate(data):
                        txt_file.write(f"  [{i:4d}]: {val}\n")
                elif len(data.shape) == 2:
                    txt_file.write(f"\n数据 (二维数组，{data.shape[0]} 行 x {data.shape[1]} 列):\n")
                    txt_file.write(f"  列索引: ")
                    if 'names' in eef_right_data:
                        names = eef_right_data['names']
                        if len(names) == data.shape[1]:
                            txt_file.write(" | ".join([f"{i:2d}:{name.decode() if isinstance(name, bytes) else name}" 
                                                      for i, name in enumerate(names)]) + "\n")
                        else:
                            txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    else:
                        txt_file.write(" | ".join([f"{i:2d}" for i in range(data.shape[1])]) + "\n")
                    txt_file.write("  " + "-" * 80 + "\n")
                    
                    for i in range(len(data)):
                        row_str = " | ".join([f"{val:12.6f}" for val in data[i]])
                        txt_file.write(f"  [{i:4d}]: {row_str}\n")
                else:
                    txt_file.write(f"\n数据 (多维数组):\n")
                    txt_file.write(f"  {data}\n")
                
                txt_file.write("\n")
                
                # 写入统计信息（仅对数值类型数据）
                if len(data.shape) >= 1 and data.size > 0:
                    # 检查是否为数值类型（排除字符串类型）
                    is_numeric = np.issubdtype(data.dtype, np.number)
                    if is_numeric:
                        txt_file.write("统计信息:\n")
                        if len(data.shape) == 1:
                            txt_file.write(f"  Min: {data.min()}\n")
                            txt_file.write(f"  Max: {data.max()}\n")
                            txt_file.write(f"  Mean: {data.mean()}\n")
                            txt_file.write(f"  Std: {data.std()}\n")
                        elif len(data.shape) == 2:
                            txt_file.write(f"  Min (每列): {data.min(axis=0)}\n")
                            txt_file.write(f"  Max (每列): {data.max(axis=0)}\n")
                            txt_file.write(f"  Mean (每列): {data.mean(axis=0)}\n")
                            txt_file.write(f"  Std (每列): {data.std(axis=0)}\n")
                        txt_file.write("\n")
        
        file_size = output_file.stat().st_size / 1024  # KB
        print(f"✅ 保存完成！")
        print(f"   文件: {output_file}")
        print(f"   大小: {file_size:.2f} KB")
        print(f"   包含 {len(eef_right_data)} 个数据集")
    else:
        print("❌ 未找到 _info_eef_right")


💾 保存 _info_eef_right 数据到 txt 文件

📂 _info_eef_right 包含的数据:
  - names: shape=(7,), dtype=|S2
  - orientation: shape=(183, 4), dtype=float64
  - position: shape=(183, 3), dtype=float64

🔄 正在读取数据...
  ✅ 已读取 names: (7,)
  ✅ 已读取 orientation: (183, 4)
  ✅ 已读取 position: (183, 3)

📋 属性: {'dof': 7, 'topic': '/info/eef_right', 'type': 'Pose'}

💾 正在保存到: pick_blue_bottle_info_eef_right.txt
✅ 保存完成！
   文件: pick_blue_bottle_info_eef_right.txt
   大小: 23.34 KB
   包含 3 个数据集


## 21. 读取新转换的 HDF5 文件 - 完整结构分析


In [ ]:
# 读取新转换的 HDF5 文件，查看完整结构
print("=" * 80)
print("📊 分析新转换的 HDF5 文件结构")
print("=" * 80)

# 使用相同的文件路径
hdf5_path_new = Path("pick_blue_bottle/rosbag2_2026_01_09-21_24_48/rosbag2_2026_01_09-21_24_48_0.h5")

print(f"\n📁 文件路径: {hdf5_path_new}")
print(f"📦 文件大小: {hdf5_path_new.stat().st_size / (1024 * 1024):.2f} MB\n")

def print_hdf5_structure(name, obj, indent=0):
    """递归打印 HDF5 文件结构"""
    prefix = "  " * indent
    if isinstance(obj, h5py.Group):
        print(f"{prefix}📁 {name}/ (Group)")
        if hasattr(obj, 'attrs') and len(obj.attrs) > 0:
            print(f"{prefix}  属性: {dict(obj.attrs)}")
        # 限制显示深度，避免过多输出
        if indent < 3:
            for key in obj.keys():
                print_hdf5_structure(key, obj[key], indent + 1)
        else:
            print(f"{prefix}  ... (共 {len(obj.keys())} 个子项)")
    elif isinstance(obj, h5py.Dataset):
        print(f"{prefix}📄 {name}: shape={obj.shape}, dtype={obj.dtype}")
        if hasattr(obj, 'attrs') and len(obj.attrs) > 0:
            attrs_str = ", ".join([f"{k}={v}" for k, v in obj.attrs.items()])
            print(f"{prefix}  属性: {attrs_str}")

with h5py.File(hdf5_path_new, 'r') as f:
    print("\n" + "=" * 80)
    print("📋 HDF5 文件完整结构:")
    print("=" * 80)
    
    # 打印根级别
    print("\n根级别:")
    for key in f.keys():
        print_hdf5_structure(key, f[key], indent=0)
    
    # 详细列出 topics（如果存在）
    print("\n" + "=" * 80)
    print("📌 Topics 详细列表:")
    print("=" * 80)
    
    if 'topics' in f:
        topics_group = f['topics']
        print(f"\n找到 'topics' 组，包含 {len(topics_group.keys())} 个 topics:\n")
        
        for topic_name in sorted(topics_group.keys()):
            topic = topics_group[topic_name]
            print(f"🔹 {topic_name}")
            
            if isinstance(topic, h5py.Group):
                print(f"   类型: Group")
                if hasattr(topic, 'attrs') and len(topic.attrs) > 0:
                    print(f"   属性: {dict(topic.attrs)}")
                print(f"   包含的数据集:")
                for key in topic.keys():
                    dataset = topic[key]
                    if isinstance(dataset, h5py.Dataset):
                        print(f"     - {key}: shape={dataset.shape}, dtype={dataset.dtype}")
            elif isinstance(topic, h5py.Dataset):
                print(f"   类型: Dataset")
                print(f"   Shape: {topic.shape}, Dtype: {topic.dtype}")
                if hasattr(topic, 'attrs') and len(topic.attrs) > 0:
                    print(f"   属性: {dict(topic.attrs)}")
            print()
    else:
        print("❌ 未找到 'topics' 组")
    
    # 列出所有顶级键
    print("\n" + "=" * 80)
    print("📌 根级别所有键:")
    print("=" * 80)
    for key in f.keys():
        obj = f[key]
        if isinstance(obj, h5py.Group):
            print(f"  📁 {key}/ - Group (包含 {len(obj.keys())} 个子项)")
        elif isinstance(obj, h5py.Dataset):
            print(f"  📄 {key} - Dataset (shape={obj.shape}, dtype={obj.dtype})")
    
    # 统计信息
    print("\n" + "=" * 80)
    print("📊 统计信息:")
    print("=" * 80)
    
    if 'topics' in f:
        topics_group = f['topics']
        total_datasets = 0
        total_size = 0
        
        for topic_name in topics_group.keys():
            topic = topics_group[topic_name]
            if isinstance(topic, h5py.Group):
                for key in topic.keys():
                    dataset = topic[key]
                    if isinstance(dataset, h5py.Dataset):
                        total_datasets += 1
                        # 估算大小（字节）
                        size = np.prod(dataset.shape) * dataset.dtype.itemsize
                        total_size += size
            elif isinstance(topic, h5py.Dataset):
                total_datasets += 1
                size = np.prod(topic.shape) * topic.dtype.itemsize
                total_size += size
        
        print(f"  Topics 数量: {len(topics_group.keys())}")
        print(f"  数据集总数: {total_datasets}")
        print(f"  数据总大小: {total_size / (1024 * 1024):.2f} MB")


## 22. 查看每个 Topic 的数据样本


In [7]:
# 查看每个 topic 的数据样本
print("=" * 80)
print("🔍 查看每个 Topic 的数据样本")
print("=" * 80)

hdf5_path_new = Path("pick_blue_bottle/rosbag2_2026_01_09-21_24_48/rosbag2_2026_01_09-21_24_48_0.h5")

with h5py.File(hdf5_path_new, 'r') as f:
    if 'topics' in f:
        topics_group = f['topics']
        
        for topic_name in sorted(topics_group.keys()):
            topic = topics_group[topic_name]
            print(f"\n{'=' * 80}")
            print(f"📌 Topic: {topic_name}")
            print(f"{'=' * 80}")
            
            if isinstance(topic, h5py.Group):
                # 显示属性
                if hasattr(topic, 'attrs') and len(topic.attrs) > 0:
                    print(f"\n属性:")
                    for attr_key, attr_value in topic.attrs.items():
                        print(f"  {attr_key}: {attr_value}")
                
                # 显示每个数据集
                for key in sorted(topic.keys()):
                    dataset = topic[key]
                    if isinstance(dataset, h5py.Dataset):
                        data = dataset[:]
                        print(f"\n  📄 {key}:")
                        print(f"    Shape: {data.shape}")
                        print(f"    Dtype: {data.dtype}")
                        
                        # 显示数据样本
                        if data.size > 0:
                            if len(data.shape) == 0:
                                # 标量
                                print(f"    值: {data}")
                            elif len(data.shape) == 1:
                                # 一维数组
                                print(f"    长度: {len(data)}")
                                if len(data) <= 10:
                                    print(f"    全部数据: {data}")
                                else:
                                    print(f"    前5个: {data[:5]}")
                                    print(f"    后5个: {data[-5:]}")
                                if np.issubdtype(data.dtype, np.number):
                                    print(f"    范围: [{data.min():.6f}, {data.max():.6f}]")
                                    print(f"    均值: {data.mean():.6f}")
                            elif len(data.shape) == 2:
                                # 二维数组
                                print(f"    维度: {data.shape[0]} 行 x {data.shape[1]} 列")
                                if data.shape[0] <= 5:
                                    print(f"    全部数据:\n{data}")
                                else:
                                    print(f"    前3行:\n{data[:3]}")
                                    print(f"    后3行:\n{data[-3:]}")
                                if np.issubdtype(data.dtype, np.number):
                                    print(f"    每列范围:")
                                    for i in range(min(5, data.shape[1])):
                                        col = data[:, i]
                                        print(f"      列{i}: [{col.min():.6f}, {col.max():.6f}], 均值: {col.mean():.6f}")
                            else:
                                # 多维数组
                                print(f"    维度: {data.shape}")
                                print(f"    前几个元素: {data.flat[:10]}")
                                if np.issubdtype(data.dtype, np.number):
                                    print(f"    范围: [{data.min():.6f}, {data.max():.6f}]")
                        else:
                            print(f"    空数组")
                            
            elif isinstance(topic, h5py.Dataset):
                # 直接是数据集
                data = topic[:]
                print(f"\n  📄 直接数据集:")
                print(f"    Shape: {data.shape}")
                print(f"    Dtype: {data.dtype}")
                
                if hasattr(topic, 'attrs') and len(topic.attrs) > 0:
                    print(f"    属性: {dict(topic.attrs)}")
                
                if data.size > 0:
                    if len(data.shape) == 0:
                        print(f"    值: {data}")
                    elif len(data.shape) == 1:
                        print(f"    长度: {len(data)}")
                        if len(data) <= 20:
                            print(f"    全部数据: {data}")
                        else:
                            print(f"    前10个: {data[:10]}")
                            print(f"    后10个: {data[-10:]}")
                        if np.issubdtype(data.dtype, np.number):
                            print(f"    范围: [{data.min():.6f}, {data.max():.6f}]")
                            print(f"    均值: {data.mean():.6f}")
                    else:
                        print(f"    维度: {data.shape}")
                        print(f"    前几个元素: {data.flat[:10]}")
                        if np.issubdtype(data.dtype, np.number):
                            print(f"    范围: [{data.min():.6f}, {data.max():.6f}]")
            
            print()
    else:
        print("❌ 未找到 'topics' 组")


🔍 查看每个 Topic 的数据样本

📌 Topic: _camera_camera_aligned_depth_to_color_image_raw

属性:
  compressed: True
  encoding: 16UC1
  fixed_length_compression: True
  max_compressed_size: 921600
  original_height: 480
  original_width: 640
  topic: /camera/camera/aligned_depth_to_color/image_raw

  📄 data:
    Shape: (183, 921600)
    Dtype: uint8
    维度: 183 行 x 921600 列
    前3行:
[[  0   0   0 ...   0   0   0]
 [255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]]
    后3行:
[[255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]
 [255 216 255 ...   0   0   0]]
    每列范围:
      列0: [0.000000, 255.000000], 均值: 241.065574
      列1: [0.000000, 216.000000], 均值: 204.196721
      列2: [0.000000, 255.000000], 均值: 241.065574
      列3: [0.000000, 224.000000], 均值: 211.759563
      列4: [0.000000, 0.000000], 均值: 0.000000

  📄 data_length:
    Shape: (183,)
    Dtype: int32
    长度: 183
    前5个: [    0 13968 14532 14784 14650]
    后5个: [16309 16344 15628 15597 15063]
    范围: [0.000000, 16665.000000]
 

## 21. 保存 gripper_feedback_R 的全部数据到 txt 文件


In [6]:
# 保存 gripper_feedback_R 的全部数据到 txt 文件
print("=" * 80)
print("💾 保存 gripper_feedback_R 数据到 txt 文件")
print("=" * 80)

output_file = Path("pick_blue_bottle_gripper_feedback_R.txt")

with h5py.File(hdf5_path, 'r') as f:
    # 尝试在 topics 组下查找
    gripper_feedback_data = None
    topic_name = None
    
    if 'topics' in f:
        # 查找可能的 topic 名称
        possible_names = ['gripper_feedback_R', '_gripper_feedback_R', 'gripper_feedbackR']
        for name in possible_names:
            if name in f['topics']:
                topic_name = name
                gripper_feedback_group = f['topics'][name]
                break
        
        if topic_name is None:
            # 列出所有 topics 以便调试
            print("\n📋 可用的 topics:")
            for key in f['topics'].keys():
                if 'gripper' in key.lower() or 'feedback' in key.lower():
                    print(f"  - {key}")
    
    if topic_name and gripper_feedback_group:
        print(f"\n📂 找到 topic: {topic_name}")
        
        # 检查数据结构
        print(f"\n📂 {topic_name} 包含的数据:")
        for key in gripper_feedback_group.keys():
            dataset = gripper_feedback_group[key]
            if isinstance(dataset, h5py.Dataset):
                print(f"  - {key}: shape={dataset.shape}, dtype={dataset.dtype}")
        
        # 读取所有数据
        print("\n🔄 正在读取数据...")
        gripper_feedback_dict = {}
        for key in gripper_feedback_group.keys():
            dataset = gripper_feedback_group[key]
            if isinstance(dataset, h5py.Dataset):
                gripper_feedback_dict[key] = dataset[:]
                print(f"  ✅ 已读取 {key}: {gripper_feedback_dict[key].shape}, dtype={gripper_feedback_dict[key].dtype}")
        
        # 读取属性
        attrs = {}
        if hasattr(gripper_feedback_group, 'attrs') and len(gripper_feedback_group.attrs) > 0:
            attrs = dict(gripper_feedback_group.attrs)
            print(f"\n📋 属性: {attrs}")
        
        # 保存到 txt 文件
        print(f"\n💾 正在保存到: {output_file}")
        with open(output_file, 'w', encoding='utf-8') as txt_file:
            # 写入头部信息
            txt_file.write("=" * 80 + "\n")
            txt_file.write("Gripper Feedback R 数据\n")
            txt_file.write("=" * 80 + "\n\n")
            txt_file.write(f"文件来源: {hdf5_path}\n")
            txt_file.write(f"保存时间: {np.datetime64('now')}\n\n")
            
            # 写入属性
            if attrs:
                txt_file.write("属性信息:\n")
                for attr_key, attr_value in attrs.items():
                    txt_file.write(f"  {attr_key}: {attr_value}\n")
                txt_file.write("\n")
            
            # 写入每个数据集
            for key, data in gripper_feedback_dict.items():
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"数据集: {key}\n")
                txt_file.write("=" * 80 + "\n")
                txt_file.write(f"Shape: {data.shape}\n")
                txt_file.write(f"Dtype: {data.dtype}\n")
                
                # 处理 bool 类型数据
                if data.dtype == bool or data.dtype == np.bool_:
                    txt_file.write(f"\n数据 (布尔数组，共 {len(data)} 个元素):\n")
                    txt_file.write("  True 表示夹爪闭合，False 表示夹爪打开\n\n")
                    for i, val in enumerate(data):
                        status = "闭合" if val else "打开"
                        txt_file.write(f"  [{i:4d}]: {val} ({status})\n")
                    
                    # 统计信息
                    txt_file.write("\n统计信息:\n")
                    true_count = np.sum(data)
                    false_count = len(data) - true_count
                    txt_file.write(f"  True (闭合) 数量: {true_count} ({true_count/len(data)*100:.2f}%)\n")
                    txt_file.write(f"  False (打开) 数量: {false_count} ({false_count/len(data)*100:.2f}%)\n")
                    txt_file.write(f"  总数量: {len(data)}\n")
                elif len(data.shape) == 1:
                    txt_file.write(f"\n数据 (一维数组，共 {len(data)} 个元素):\n")
                    for i, val in enumerate(data):
                        txt_file.write(f"  [{i:4d}]: {val}\n")
                elif len(data.shape) == 2:
                    txt_file.write(f"\n数据 (二维数组，{data.shape[0]} 行 x {data.shape[1]} 列):\n")
                    txt_file.write("  " + "-" * 80 + "\n")
                    for i in range(len(data)):
                        row_str = " | ".join([f"{val:12.6f}" if isinstance(val, (int, float)) else str(val) for val in data[i]])
                        txt_file.write(f"  [{i:4d}]: {row_str}\n")
                else:
                    txt_file.write(f"\n数据 (多维数组):\n")
                    txt_file.write(f"  {data}\n")
                
                txt_file.write("\n")
        
        file_size = output_file.stat().st_size / 1024  # KB
        print(f"✅ 保存完成！")
        print(f"   文件: {output_file}")
        print(f"   大小: {file_size:.2f} KB")
        print(f"   包含 {len(gripper_feedback_dict)} 个数据集")
    else:
        print("❌ 未找到 gripper_feedback_R")
        print("\n💡 提示: 请检查 HDF5 文件中的 topics 组，查找包含 'gripper' 或 'feedback' 的 topic 名称")


💾 保存 gripper_feedback_R 数据到 txt 文件

📂 找到 topic: _gripper_feedback_R

📂 _gripper_feedback_R 包含的数据:
  - data: shape=(183, 5), dtype=float32
  - names: shape=(5,), dtype=|S9

🔄 正在读取数据...
  ✅ 已读取 data: (183, 5), dtype=float32
  ✅ 已读取 names: (5,), dtype=|S9

📋 属性: {'dof': 5, 'topic': '/gripper/feedback_R', 'type': 'Float32MultiArray', 'variable_length': False}

💾 正在保存到: pick_blue_bottle_gripper_feedback_R.txt
✅ 保存完成！
   文件: pick_blue_bottle_gripper_feedback_R.txt
   大小: 12.46 KB
   包含 2 个数据集
